<a href="https://colab.research.google.com/github/harikang/yai_implementation/blob/harikang-patch-1/Transformer_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer

In [ ]:
import torch
from torch import nn

# Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
   def __init__(self, d_model, max_len, device):
        super(PositionalEncoding, self).__init__()
        # positional encoding은 input embedding과 elementwise더해져야한다.
        # encoding = [단어의 개수 x 단어 별 차원]
        self.encoding = torch.zeros(max_len, d_model, device=device)
        self.encoding.requires_grad = False  # we don't need to compute gradient

        pos = torch.arange(0, max_len, device=device)#0부터 max_length까지 1씩 증가하여 쌓임
        pos = pos.float().unsqueeze(dim=1) # pos = [max_length x 1]차원
       #0부터 d_model까지 2 간격으로 순서대로 쌓임
        _2i = torch.arange(0, d_model, step=2, device=device).float()
        # 'i' means index of d_model (e.g. embedding size = 50, 'i' = [0,50])
        # "step=2" means 'i' multiplied with two (same with 2 * i)
        #아래 코드가 PE 코드를 의미하며 PE[단어의 개수 X 단어의 차원]
        #단어 별 FEATURE가 짝수일 때,
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        #단어 별 FEATURE가 홀수일 때,
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))
        # compute positional encoding to consider positional information of words

    def forward(self, x):
        batch_size, seq_len = x.size()
        # [batch_size = 문장의 개수, seq_len = 문장 별 단어의 최대 개수]
        return self.encoding[:seq_len, :]
        # [max_len, 단어의 차원]
        #단어의 특징을 나타내는 차원마다 주파수가 고정된다.
        #단어의 순서에 따라 pos값이 변하면서 다른 값으로 지정된다.
        #즉, 다른 문장의 같은 위치의 특정 차원은 같은 값을 가짐

# Input Embedding